In [83]:
import pandas as pd
import numpy as np
import os

In [84]:
path = '/mnt/c/Users/vishal.nanaware/Documents/model_building/naved_pending_work/MR_PYT_045_AmortBond_YTM/data/MR_PYT_045_AmortBond_YTM.xlsx'

In [85]:
#reading the input data
exposure = pd.read_excel(path,sheet_name = 'Exposure')
rates_ytm = pd.read_excel(path,sheet_name = 'rates_ytm')
rates_spread = pd.read_excel(path,sheet_name = 'rates_spread')
rates_interest = pd.read_excel(path,sheet_name = 'rates_interest')
rates_exchange = pd.read_excel(path,sheet_name = 'rates_exchange')

In [86]:
def data_sort(df):
    df = df.dropna(axis=1,how = 'all')
    df = df.dropna(how = 'all')
    df.columns = df.columns.str.replace(" ","_")
    df.columns = df.columns.str.lower()
    return df

In [87]:
exposure = data_sort(exposure)
rates_ytm = data_sort(rates_ytm)
rates_spread = data_sort(rates_spread)
rates_interest = data_sort(rates_interest)
rates_exchange = data_sort(rates_exchange)

In [88]:
mtm = pd.DataFrame()
exposure['transaction_id'].unique()

array(['INE261F901Y5Dealer1AFS', 'INE261F901Y5_1Dealer1AFS'], dtype=object)

In [126]:
for k,h in exposure.groupby('transaction_id'):
    h['residual_maturity'] = (round((h[h['cashflow_type'] == 'P']['residual_maturity_actual']).sum()*(h[h['cashflow_type'] == 'P'].reset_index().iloc[0]['cashflow_amount']/h[h['cashflow_type'] == 'P'].reset_index().iloc[0]['remaining_notional_amt'])))
    h['fixed_floating'] = 'F'
    h['cashflow_amount'] = (h[h['cashflow_type'] == 'P']['residual_maturity_actual']).sum()
    h['interpolated_curve_rate'] = rates_ytm[rates_ytm['instrument'] == h['instrument_code'].values[0]]['ytm'].values[0]
    #h['residual_maturity_in_year'] = h['residual_maturity']/365
    h = current_rates_spread(h,rates_spread)
    h['spread_lower_maturity_days'] = h['spread_lower_tenor']*365
    h['spread_upper_maturity_days'] = h['spread_upper_tenor']*365
    h['spread_lower_tenor_rate'] = (rates_spread[(rates_spread['rating_label'] == h['rating'].values[0]) & (rates_spread['as_on_date'] == h['as_on_date'].values[0]) & (rates_spread['industry_segment'] == h['industry_segment'].values[0]) & (rates_spread['tenor_code'] == h['spread_lower_tenor'].values[0])]['spread_value(in_bps)'].values[0]/100)
    h['spread_upper_tenor_rate'] = (rates_spread[(rates_spread['rating_label'] == h['rating'].values[0]) & (rates_spread['as_on_date'] == h['as_on_date'].values[0]) & (rates_spread['industry_segment'] == h['industry_segment'].values[0]) & (rates_spread['tenor_code'] == h['spread_upper_tenor'].values[0])]['spread_value(in_bps)'].values[0]/100)
    print('@@@@@@@@@@@@@@')
    h = interpolated_rate_calculation(h)
    #print(h.info())
    #print(h.info())
    print(h[['spread_lower_tenor','spread_upper_tenor','spread_lower_tenor_rate','spread_upper_tenor_rate','residual_maturity','spread_lower_maturity_days','spread_upper_maturity_days']])
    print('!!!!!!!!!!!!!!!')
    
    
                              


@@@@@@@@@@@@@@
    spread_lower_tenor  spread_upper_tenor  spread_lower_tenor_rate   
0                  3.0                 4.0                 4.456541  \
1                  3.0                 4.0                 4.456541   
2                  3.0                 4.0                 4.456541   
3                  3.0                 4.0                 4.456541   
4                  3.0                 4.0                 4.456541   
5                  3.0                 4.0                 4.456541   
6                  3.0                 4.0                 4.456541   
7                  3.0                 4.0                 4.456541   
8                  3.0                 4.0                 4.456541   
9                  3.0                 4.0                 4.456541   
10                 3.0                 4.0                 4.456541   
11                 3.0                 4.0                 4.456541   
12                 3.0                 4.0                 4.4

In [92]:
def current_rates_spread(df,spread):

    maturity_in_years = float( df['residual_maturity'].iloc[0] /365 )


    if (pd.isna(df['industry_segment'].iloc[0])== False) & (pd.isna(df['rating'].iloc[0])== True):
        
        rating = 'BBB-'  
        
        df['spread_key'] = df['industry_segment'] + "_"+ rating

        current_spreads = spread.loc[spread['as_on_date'].isin(df['as_on_date'])].reset_index(drop=True)

        current_spreads = current_spreads.loc[current_spreads['industry_segment'].isin(df['industry_segment'])].reset_index(drop=True)
       
        current_spreads_1 =current_spreads.loc[current_spreads['rating_label'] == rating].reset_index(drop=True)
        
        current_spreads_1['spread_value(in_bps)'] = current_spreads_1['spread_value(in_bps)'] /100
        spread_tenor = current_spreads_1['tenor_code']
        
        for index in range((len(spread_tenor))):
            if maturity_in_years <= spread_tenor[0] :
                # print(spread_tenor[0])
                df['spread_lower_tenor'] = spread_tenor[0]
                df['spread_upper_tenor'] = spread_tenor[0]
                df['spread_lower_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][0]
                df['spread_upper_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][0]

            elif maturity_in_years > spread_tenor[len(spread_tenor)-1]:
                # print(spread_tenor[0])
                df['spread_lower_tenor'] = spread_tenor[len(spread_tenor)]
                df['spread_upper_tenor'] = spread_tenor[len(spread_tenor)]
                df['spread_lower_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][len(spread_tenor)]
                df['spread_upper_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][len(spread_tenor)]

            elif maturity_in_years > spread_tenor[index] and maturity_in_years < spread_tenor[index+1]:
                # print(spread_tenor[index])

                df['spread_lower_tenor'] = spread_tenor[index]
                df['spread_upper_tenor'] = spread_tenor[index+1]
                df['spread_lower_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][index]
                df['spread_upper_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][index+1]

            elif maturity_in_years == spread_tenor[index]:
                # print(spread_tenor[index])

                df['spread_lower_tenor'] = spread_tenor[index]
                df['spread_upper_tenor'] = spread_tenor[index]
                df['spread_lower_tenor_rate'] =current_spreads_1['spread_value(in_bps)'][index]
                df['spread_upper_tenor_rate'] =current_spreads_1['spread_value(in_bps)'][index]
    
    elif pd.isna(df['industry_segment'].iloc[0]) & pd.isna(df['rating'].iloc[0]) is False:
        
        df['spread_key'] = df['industry_segment'] + "_"+ df['rating']
       
        spread['key'] = spread['industry_segment'] + "_" + spread['rating_label']
        
        current_spreads = spread.loc[spread['as_on_date'].isin(df['as_on_date'])].reset_index(drop=True)
        current_spreads_1 =current_spreads.loc[current_spreads['key'].isin(df['spread_key'])].reset_index(drop=True)
        
        current_spreads_1['spread_value(in_bps)'] = current_spreads_1['spread_value(in_bps)'] /100

        spread_tenor = current_spreads_1['tenor_code']

        for index in range((len(spread_tenor))):

            if maturity_in_years <= spread_tenor[0] :

                df['spread_lower_tenor'] = spread_tenor[0]
                df['spread_upper_tenor'] = spread_tenor[0]
                df['spread_lower_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][0]
                df['spread_upper_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][0]

            elif maturity_in_years > spread_tenor[len(spread_tenor)-1]:

                df['spread_lower_tenor'] = spread_tenor[len(spread_tenor)]
                df['spread_upper_tenor'] = spread_tenor[len(spread_tenor)]
                df['spread_lower_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][len(spread_tenor)]
                df['spread_upper_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][len(spread_tenor)]

            elif maturity_in_years > spread_tenor[index] and maturity_in_years < spread_tenor[index+1]:

                df['spread_lower_tenor'] = spread_tenor[index]
                df['spread_upper_tenor'] = spread_tenor[index+1]
                df['spread_lower_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][index]
                df['spread_upper_tenor_rate'] = current_spreads_1['spread_value(in_bps)'][index+1]

            elif maturity_in_years == spread_tenor[index]:

                df['spread_lower_tenor'] = spread_tenor[index]
                df['spread_upper_tenor'] = spread_tenor[index]
                df['spread_lower_tenor_rate'] =current_spreads_1['spread_value(in_bps)'][index]
                df['spread_upper_tenor_rate'] =current_spreads_1['spread_value(in_bps)'][index]

    else:
        df['spread_key'] = "-"
        df['spread_lower_tenor'] = 0
        df['spread_upper_tenor'] = 0
        df['spread_lower_tenor_rate'] = 0
        df['spread_upper_tenor_rate'] = 0
    
    df['spread_lower_maturity_days'] =df['spread_lower_tenor']*365
    df['spread_upper_maturity_days'] =df['spread_upper_tenor']*365

    return df


In [123]:
def interpolated_rate_calculation(temp_output):
    """
    Input : temp_output : dataframe which contain the column lower_rates and upper_rates
    Output : temp_output : dataframe which contain column interpolated_rates
    """
    
    if temp_output['spread_lower_tenor'].values[0]==temp_output['spread_upper_tenor'].values[0]:
        temp_output['interpolated_spread_rate']=round(temp_output['spread_lower_tenor_rate'].values[0],9)
    else:
        temp_output['interpolated_spread_rate']=round(temp_output['spread_lower_tenor_rate'].values[0]+((temp_output['spread_upper_tenor_rate'].values[0]-temp_output['spread_lower_tenor_rate'].values[0])*(temp_output['residual_maturity'].values[0]-temp_output['spread_lower_maturity_days'].values[0])/(temp_output['spread_upper_maturity_days'].values[0]-temp_output['spread_lower_maturity_days'].values[0])),9)
    
    return temp_output

In [128]:
rates_spread['spread_value(in_bps)']

0      562.496969
1      561.496969
2      561.496969
3      561.496969
4      562.496969
          ...    
706    316.821987
707    314.770276
708    368.255153
709    360.377998
710    338.691816
Name: spread_value(in_bps), Length: 711, dtype: float64